# AgentRun Usage 

Now, let's see AgentRun in action with something more complicated. We will take advantage of function calling and AgentRun, to have LLMs write and execute code on the fly to solve arbitrary tasks. You can find the full code under `docs/examples/`

> We are using the REST API as it is recommend to seperate the code execution service from the rest of our infrastructure.

First, we will install the needed packages. We are using mixtral here via groq to keep things fast and with minimal depenencies, but AgentRun works with any LLM out of the box. All what's required is for the LLM to return a code snippet.

> FYI: OpenAI assistant tool ` code_interpreter` can execute code. Agentrun is a transparent, open-source version that can work with any LLM.

In [1]:
# Agentrun Example
!pip install requests
!pip install groq

  Using cached groq-0.5.0-py3-none-any.whl.metadata (12 kB)
  Using cached anyio-4.3.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached pydantic-2.7.0-py3-none-any.whl.metadata (103 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached typing_extensions-4.11.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached annotated_types-0.6.0-py3-none-any.whl.metadata (12 kB)
Using cached groq-0.5.0-py3-none-any.whl (75 kB)
Using cached anyio-4.3.0-py3-none-any.whl (85 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
Using cached httpcore-1.0.5-py3-none-any.whl (77 kB)
Using cached pydantic-2.7.0-py3-none-any.whl (407 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

Next, we will setup a function that executed the code and returns an output. We are using the API here, so make sure to have it running before trying this. 

Here is the steps to run the API:
1. git clone https://github.com/Jonathan-Adly/agentrun
2. cd agentrun/agentrun-api
3. cp .env.example .env.dev
4. docker-compose up -d --build

In [2]:
import requests
def execute_python_code(code: str) -> str:
    response = requests.post("http://localhost:8000/v1/run/", json={"code": code})
    output = response.json()["output"]
    return output

execute_python_code("print('Hello, World!')")

'Hello, World!\n'

Next, we will setup our LLM function calling skeleton code. We need:

1. An LLM client such Groq or OpenAI or Anthropic (alternatively, you can use liteLLm as wrapper)
2. The model you will use 
3. Our code execution tool - that encourages the LLM model to send us python code to execute reliably

In [3]:
from groq import Groq
import json

client = Groq(api_key ="Your API Key")

MODEL = 'mixtral-8x7b-32768'

tools = [
    {
        "type": "function",
        "function": {
            "name": "execute_python_code",
            "description": "Sends a python code snippet to the code execution environment and returns the output. The code execution environment can automatically import any library or package by importing.",
            "parameters": {
                "type": "object",
                "properties": {
                    "code": {
                        "type": "string",
                        "description": "The code snippet to execute. Must be a valid python code. Must use print() to output the result.",
                    },
                },
                "required": ["code"],
            },
        },
    },
]

Next, we will setup a function to call our LLM of choice.

In [4]:
def chat_completion_request(messages, tools=None, tool_choice="auto", model=MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

Finally, we will set up a function that takes the user query and returns an answer. Using Agentrun to execute code when the LLM determines code execution is necesary to answer the question

In [5]:
def get_answer(query):
    messages = []
    messages.append(
        {
            "role": "system",
            "content": """Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.\n 
            Use the execute_python_code tool to run code if a question is better solved with code. You can use any package in the code snippet by simply importing. Like `import requests` would work fine.\n
            """,
        }
    )
    messages.append({"role": "user", "content": query})

    chat_response = chat_completion_request(messages, tools=tools)

    message = chat_response.choices[0].message
    # tool call versus content
    if message.tool_calls:
        tool_call = message.tool_calls[0]
        arg = json.loads(tool_call.function.arguments)["code"]
        print(f"Executing code: {arg}")
        answer = execute_python_code(arg)
        # Optional: call an LLM again to turn the answer to a human friendly response
        query = "Help translate the code output to a human friendly response. This was the user query: " + query + " The code output is: " + answer
        answer = get_answer(query)
    else:
        answer = message.content

    return answer

Now let's try it!

In [6]:
get_answer("what's the average daily move of Apple stock in the last 3 days?")

Executing code: import yfinance as yf

stock = yf.Ticker('AAPL')
historical_data = stock.history(period='3d')

# Calculate the daily move as the percentage change from close to close
daily_moves = [(historical_data['Close'][i] - historical_data['Close'][i-1]) / historical_data['Close'][i-1] for i in range(1, len(historical_data['Close']))]

# Calculate the average daily move
average_daily_move = sum(daily_moves) / len(daily_moves) * 100

print(average_daily_move)


"The average daily movement of Apple's stock in the last 3 days is approximately $2.60."